# モデルの保存と復元

モデルは訓練中にも、訓練が終わった後も保存できる。
訓練を中断したところから再開したり、他人とモデルを共有するのに役立つ。

研究成果であるモデルや手法を公開する際、以下を共有するのが一般的。

- モデルを構築するプログラム
- 学習済みのモデルの重みあるいはパラメータ


今回のチュートリアルでは、`tf.keras`を使った場合のモデルの保存を実行する。

(Kerasではない)別のAPIを使う場合など、他のアプローチの場合は、以下を参照。  
- [Save and Restore](https://www.tensorflow.org/guide/saved_model?hl=ja)
- [Saving in eager](https://www.tensorflow.org/guide/eager?hl=ja#object-based_saving)

In [18]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.1.0'

In [19]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## モデルの定義

重みの保存と読み込みのデモを行うための簡単なモデルを定義

In [20]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation="relu", input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation="softmax")
    ])
    
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

# 基本的なモデルのインスタンスを作る
model = create_model()
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 訓練中にチェックポイントを保存する

`tf.keras.callbacks.ModelCheckpoint`コールバックを使用すると、チェックポイントを自動的に保存することができる。
再び訓練を行うことなくモデルを使用したり、訓練が中断された際に、途中から再開できる。

### チェックポイントコールバックの使い方

モデルの訓練時に、`ModelCheckpoint`を渡す。

In [21]:
checkpoint_path = "training_1/cp.ckpt"
checkpint_dir = os.path.dirname(checkpoint_path)

# チェックポイントコールバックを作る
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model = create_model()

model.fit(train_images, train_labels, epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback]) # 訓練にコールバックを渡す

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 576/1000 [================>.............] - ETA: 0s - loss: 1.4959 - accuracy: 0.5729
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 506us/sample - loss: 1.1571 - accuracy: 0.6710 - val_loss: 0.7703 - val_accuracy: 0.7500
Epoch 2/10
 576/1000 [================>.............] - ETA: 0s - loss: 0.4356 - accuracy: 0.8646
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 168us/sample - loss: 0.4277 - accuracy: 0.8770 - val_loss: 0.5137 - val_accuracy: 0.8480
Epoch 3/10
 608/1000 [=================>............] - ETA: 0s - loss: 0.3212 - accuracy: 0.9145
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 170us/sample - loss: 0.2962 - accuracy: 0.9230 - val_loss: 0.4717 - val_accuracy: 0.8510
Epoch 4/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.2117 - accuracy: 0.9526
Epoch

In [22]:
!ls {checkpint_dir}

'ls' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B


新しいモデルを作成する。ただし、保存した重みをロードできるように同じアーキテクチャになっている必要がある。(関数を使って同じアーキテクチャを作成すればよい。)

まずは学習をせずに評価を行う。続いて、チェックポイントから重みをロードしてから評価を行う。

In [23]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}".format(100*acc))

1000/1000 - 0s - loss: 2.3685 - accuracy: 0.0710
Untrained model, accuracy:  7.10


In [24]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}".format(100*acc))

1000/1000 - 0s - loss: 0.4215 - accuracy: 0.8680
Restored model, accuracy: 86.80


### チェックポイントコールバックのオプション

このコールバックには、チェックポイントに一意な名前をつけたり、チェックポイントの頻度を調整するためのオプションがある。

新しいモデルを訓練し、5エポックごとに一意な名前のチェックポイントを保存する。

In [25]:
# ファイル名に(`str.format`を使って)エポック数を埋め込みます
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 重みを5エポックごとに保存します
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [26]:
! ls {checkpoint_dir}

'ls' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B


In [27]:
# 最新のチェックポイントは以下の関数で取得できる
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [28]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4846 - accuracy: 0.8760
Restored model, accuracy: 87.60%


## ここまでのまとめ

上記のコードでは、重みだけをバイナリで`checkpoint`形式の一連のファイルで保存していた。
このチェックポイントには、以下が含まれている。

- 1つ以上のモデルの重みの断片
- どの重みがどの断片に保存されているかを示すインデックスファイル

## 手動で重みを保存する

手動で重みを保存することも可能。`Model.save_weights`メソッドを使用する。

In [29]:
model.save_weights("./checkpoints/my_checkpoint")

model = create_model()
model.load_weights("./checkpoints/my_checkpoint")

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4846 - accuracy: 0.8760
Restored model, accuracy: 87.60%


## モデル全体の保存

モデルとオプティマイザを、i)その状態（重みと変数）とii)モデルの設定の両方を含む1つのファイルに保存することができる。
オプティマイザの状態が復元されるので、中断したところから再開できる。

モデル全体を保存しておくと、TensorFlow.jsやTensorFlow Liteへの流用も可能。
- https://www.tensorflow.org/js/tutorials/conversion/import_keras?hl=ja
- https://www.tensorflow.org/js/tutorials/conversion/import_saved_model?hl=ja
- https://www.tensorflow.org/lite/convert/python_api?hl=ja#exporting_a_tfkeras_file_
- https://www.tensorflow.org/lite/convert/python_api?hl=ja#exporting_a_savedmodel_

### HDF5ファイルとして

Kerasでは、HDF55標準を使った基本的なファイルフォーマットが利用できる。
保存されたモデルは単独のバイナリナージオブジェクト(blob)として扱うことができる。

In [30]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

model.save("my_model.h5")

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 373us/sample - loss: 1.1515 - accuracy: 0.6890
Epoch 2/5
1000/1000 [==============================] - 0s 90us/sample - loss: 0.4261 - accuracy: 0.8810
Epoch 3/5
1000/1000 [==============================] - 0s 96us/sample - loss: 0.2866 - accuracy: 0.9270
Epoch 4/5
1000/1000 [==============================] - 0s 87us/sample - loss: 0.2159 - accuracy: 0.9440
Epoch 5/5
1000/1000 [==============================] - 0s 88us/sample - loss: 0.1596 - accuracy: 0.9670


In [31]:
# 重みとオプティマイザを含む全く新しいモデルを再作成
new_model = keras.models.load_model("my_model.h5")
new_model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [32]:
# 正解率を検査
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4186 - accuracy: 0.8650
Restored model, accuracy: 86.50%


この方法では、次のすべてが保存される。

- 重みの値
- モデルの設定 (アーキテクチャ)
- オプティマイザの設定

Kerasは保存する際にアーキテクチャを調べます。いまのところ、TensorFlowのオプティマイザ（tf.trainに含まれるもの）を保存することはできません。TensorFlowのオプティマイザを使用している場合には、モデルをロードしたあと再コンパイルする必要があり、オプティマイザの状態は失われます。

### saved_modelとして

~~この手法による`tf.keras`モデルの保存は実験的なもので、将来のバージョンで変更される可能性がある。~~
既にdeprecatedになっているため、以降のチュートリアルは割愛。